<a href="https://colab.research.google.com/github/bharath2213/Projects/blob/main/AARD_PROJECT_2_EDA(cleaning_Process).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install and import required libraries
!pip install pandas numpy matplotlib seaborn --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns




In [ ]:
from google.colab import files
uploaded = files.upload()   # upload scarce_items_analysis.csv

df = pd.read_csv("scarce_items_analysis.csv")


Saving scarce_items_analysis.csv to scarce_items_analysis (1).csv


In [ ]:


df = pd.read_csv("scarce_items_analysis.csv")

In [ ]:
df.head()



,Item_ID,Category,Region,Date,Stock_Level,Demand,Supply,Is_Scarce,Potentially_Scarce,Conversion_Rate,Experience_Score,Dispute_Amount,Win_Amount,Recouped_Amount,Tag_Status,Net_Loss,Spike_Indicator
0,ITEM00000,Home Appliances,South,2023-07-27,270,623,71,No,No,0.31,3,162.62,185.71,29.37,Reviewed,-52.46,No
1,ITEM00001,Toys,West,2023-07-29,391,913,644,No,Yes,0.48,6,328.36,227.56,85.72,Flagged,15.08,No
2,ITEM00002,Groceries,Central,2023-01-01,431,148,373,No,No,0.15,7,417.27,420.27,219.58,Resolved,-222.58,No
3,ITEM00003,Toys,West,2023-12-26,226,353,566,Yes,Yes,0.24,3,431.93,17.13,226.70,Reviewed,188.10,No
4,ITEM00004,Toys,West,2023-03-06,471,958,310,No,No,0.27,3,437.88,253.94,97.37,Escalated,86.57,No


In [ ]:
df.shape          # number of rows and columns
df.info()         # datatypes and missing values
df.describe()     # numerical summary
df.columns        # list all column names


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Item_ID             10000 non-null  object 
 1   Category            10000 non-null  object 
 2   Region              10000 non-null  object 
 3   Date                10000 non-null  object 
 4   Stock_Level         10000 non-null  int64  
 5   Demand              10000 non-null  int64  
 6   Supply              10000 non-null  int64  
 7   Is_Scarce           10000 non-null  object 
 8   Potentially_Scarce  10000 non-null  object 
 9   Conversion_Rate     10000 non-null  float64
 10  Experience_Score    10000 non-null  int64  
 11  Dispute_Amount      10000 non-null  float64
 12  Win_Amount          10000 non-null  float64
 13  Recouped_Amount     10000 non-null  float64
 14  Tag_Status          10000 non-null  object 
 15  Net_Loss            10000 non-null  float64
 16  Spike

Index(['Item_ID', 'Category', 'Region', 'Date', 'Stock_Level', 'Demand',
       'Supply', 'Is_Scarce', 'Potentially_Scarce', 'Conversion_Rate',
       'Experience_Score', 'Dispute_Amount', 'Win_Amount', 'Recouped_Amount',
       'Tag_Status', 'Net_Loss', 'Spike_Indicator'],
      dtype='object')

In [ ]:
# Remove duplicate records
df.drop_duplicates(inplace=True)

# Check null values
df.isnull().sum().sort_values(ascending=False)


,0
Item_ID,0
Category,0
Region,0
Date,0
Stock_Level,0
Demand,0
Supply,0
Is_Scarce,0
Potentially_Scarce,0
Conversion_Rate,0


In [ ]:
# Numeric columns
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

# Categorical columns
cat_cols = df.select_dtypes(exclude=[np.number]).columns
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])


In [ ]:
# Strip spaces from column names
df.columns = df.columns.str.strip()

# Strip spaces inside text values
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Convert date column if present
if 'Date' in df.columns or 'date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Convert categorical flags
for col in ['is_scarce', 'potentially_scarce', 'spike_indicator']:
    if col in df.columns:
        df[col] = df[col].astype('category')


/tmp/ipython-input-3139124794.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
# 1. Supply-Demand Ratio
if {'supply', 'demand'}.issubset(df.columns):
    df['supply_demand_ratio'] = df['supply'] / (df['demand'] + 1e-6)

# 2. Scarce / Potentially Scarce Tagging
if {'stock_level', 'demand'}.issubset(df.columns):
    df['is_scarce'] = np.where(df['stock_level'] < df['demand'] * 0.5, 'Yes', 'No')
    df['potentially_scarce'] = np.where(df['supply_demand_ratio'].between(0.5, 1.0), 'Yes', 'No')

# 3. Net Loss Calculation
if {'dispute_amount','win_amount','recouped_amount'}.issubset(df.columns):
    df['net_loss'] = df['dispute_amount'] - df['win_amount'] - df['recouped_amount']

# 4. Spike Indicator
if 'net_loss' in df.columns:
    df['spike_indicator'] = np.where(df['net_loss'] > 700, 'Spike', 'Normal')

# 5. Month-Year for trend visuals
if 'date' in df.columns:
    df['month_year'] = df['date'].dt.to_period('M').astype(str)


In [ ]:
print(df.info())
print(df.describe())
print(df.head(10))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Item_ID             10000 non-null  object        
 1   Category            10000 non-null  object        
 2   Region              10000 non-null  object        
 3   Date                10000 non-null  datetime64[ns]
 4   Stock_Level         10000 non-null  int64         
 5   Demand              10000 non-null  int64         
 6   Supply              10000 non-null  int64         
 7   Is_Scarce           10000 non-null  object        
 8   Potentially_Scarce  10000 non-null  object        
 9   Conversion_Rate     10000 non-null  float64       
 10  Experience_Score    10000 non-null  int64         
 11  Dispute_Amount      10000 non-null  float64       
 12  Win_Amount          10000 non-null  float64       
 13  Recouped_Amount     10000 non-null  float64    

In [ ]:
df.to_csv("cleaned_scarce_items_analysis.csv", index=False)

from google.colab import files
files.download("cleaned_scarce_items_analysis.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>